In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('data/transformed_dfs/Proposicoes_2013_2024.csv')

In [6]:
df

,nomeProposicao,idProposicao,partidoAutor,DataApresentacao,ano,Administração e Políticas Públicas,Desenvolvimento Social e Urbano,Direitos e Justiça,Economia e Desenvolvimento Sustentável,Infraestrutura e Gestão de Recursos,VirouLei
0,PL 7021/2013,604997,PSD,2013-12-19,2013,0,0,1,0,0,0
1,PL 7020/2013,604994,PRP,2013-12-19,2013,0,0,0,1,0,0
2,PL 7017/2013,604974,PSC,2013-12-18,2013,0,0,0,1,1,0
3,PL 7016/2013,604968,PMDB,2013-12-18,2013,1,0,0,0,0,0
4,PL 7014/2013,604961,PROS,2013-12-18,2013,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
8061,PL 636/2024,2419837,CIDADANIA,2024-03-07,2024,0,1,0,0,0,0
8062,PL 590/2024,2419509,REPUBLIC,2024-03-06,2024,0,0,1,0,0,0
8063,PL 545/2024,2419197,PT,2024-03-05,2024,1,0,0,1,0,1
8064,PL 147/2024,2417234,SOLIDARI,2024-02-06,2024,0,0,1,0,0,1


In [8]:
df[df['partidoAutor'].str.strip() == 'PT']

,nomeProposicao,idProposicao,partidoAutor,DataApresentacao,ano,Administração e Políticas Públicas,Desenvolvimento Social e Urbano,Direitos e Justiça,Economia e Desenvolvimento Sustentável,Infraestrutura e Gestão de Recursos,VirouLei
5,PL 7004/2013,604881,PT,2013-12-18,2013,1,0,1,0,1,0
17,PL 6982/2013,604642,PT,2013-12-17,2013,0,0,1,0,0,0
19,PL 6979/2013,604627,PT,2013-12-17,2013,0,1,0,0,0,0
22,PL 6960/2013,604444,PT,2013-12-13,2013,1,0,0,0,0,0
30,PL 6945/2013,604214,PT,2013-12-11,2013,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
8048,PL 3027/2024,2450707,PT,2024-08-02,2024,0,0,0,1,1,1
8052,PL 1725/2024,2432767,PT,2024-05-09,2024,0,0,0,1,0,1
8058,PL 1026/2024,2424079,PT,2024-03-27,2024,0,0,0,1,0,1
8063,PL 545/2024,2419197,PT,2024-03-05,2024,1,0,0,1,0,1


Teste com Cost-Sensitive Learning

In [6]:
colunas = df.columns.tolist()
X = df[colunas[5:-2]]
y = df[colunas[-1]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Aplicando o class_weight como balanced para lidar com o desbalanceamento
clf = DecisionTreeClassifier(class_weight='balanced')
clf.fit(X_train, y_train)

# Fazendo previsões
y_pred = clf.predict(X_test)

# Relatório de classificação
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.93      0.66      0.78      2184
           1       0.15      0.55      0.24       236

    accuracy                           0.65      2420
   macro avg       0.54      0.61      0.51      2420
weighted avg       0.86      0.65      0.72      2420

